# Agentic & MCP Demo



## Setup

### Pre-Requisites

Before starting, ensure you have the following:
- A running Llama Stack server with the mcp::crm toolgroup configured

### Installing dependencies

This code requires `llama-stack` and the `llama-stack-client`, both at version `0.2.2`. Lets begin by installing them:

In [1]:
!pip3 install llama-stack-client==0.2.2 llama-stack==0.2.2


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip


### Configuring logging

Now that we have our dependencies, lets setup logging for the application:

In [ ]:
from llama_stack_client.lib.agents.event_logger import EventLogger

import logging

logger = logging.getLogger(__name__)
if not logger.hasHandlers():  
    logger.setLevel(logging.INFO)
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(message)s')
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

### Connecting to llama-stack server

For the llama-stack instance, you can either run it locally or connect to a remote llama-stack instance.


In [ ]:
import sys
import os
# Add project root (assumes notebook is in ./notebooks)
sys.path.append(os.path.abspath(".."))
print("sys.path now includes:", sys.path[-1])
from dotenv import load_dotenv
from scripts.utils import get_any_available_model
load_dotenv()

base_url = os.getenv("REMOTE_BASE_URL", "http://localhost:5001")

from llama_stack_client import LlamaStackClient

client = LlamaStackClient(
    base_url=base_url
)
model = get_any_available_model(client)
# model = 'granite-3-8b-instruct'

print(model)
    
logger.info(f"Connected to Llama Stack server @ {base_url} \n")

Connected to Llama Stack server @ http://localhost:5001 



sys.path now includes: /Users/phayes/projects/rh-summit-agentic-demo
llama32-3b


### Validate tools are available in our llama-stack instance

When an instance of llama-stack is redeployed your tools need to re-registered. Also if a tool is already registered with a llama-stack instance, if you try to register one with the same `toolgroup_id`, llama-stack will throw you an error.

For this reason it is recommended to include some code to validate your tools and toolgroups. This is where the `mcp_url` comes into play. The following code will check that  the `builtin::crm` toolgroups are registered as tools.

In [ ]:
registered_tools = client.tools.list()
registered_toolgroups = [t.toolgroup_id for t in registered_tools]
logger.info(f"Your Llama Stack server is already registered with the following tool groups @ {set(registered_toolgroups)} \n")

Your Llama Stack server is already registered with the following tool groups @ {'mcp::crm', 'builtin::rag', 'builtin::code_interpreter', 'builtin::websearch'} 



## Get active opportunities

This is a simple example to get a list of active opportunities from the CRM system.

### System Prompts for different models

**Note:** If you have multiple models configured with your Llama Stack server, you can choose which one to run your queries against. When switching to a different model, you may need to adjust the system prompt to align with that model’s expected behavior. Many models provide recommended system prompts for optimal and reliable outputs—these are typically documented on their respective websites.

In [11]:
# Here is a system prompt we have come up with which works well for this query

sys_prompt1= """You are a helpful assistant. Use tools to answer. When you use a tool always respond with a summary of the result."""

In [17]:
print(model)
from llama_stack_client import Agent
# Create simple agent with tools
agent = Agent(
    client,
    model=model, # replace this with your choice of model
    instructions = sys_prompt1 , # update system prompt based on the model you are using
    tools=["mcp::crm"],
    tool_config={"tool_choice":"auto"},
    sampling_params={
        "max_tokens":4096,
        "strategy": {"type": "greedy"},
    }
)

user_prompts = ["Get active opportunities for acme"]
session_id = agent.create_session(session_name="crm_demo")

for prompt in user_prompts:
    turn_response = agent.create_turn(
        messages=[
            {
                "role":"user",
                "content": prompt
            }
        ],
        session_id=session_id,
        stream=True,
    )
    for log in EventLogger().log(turn_response):
        log.print()

llama32-3b
inference> 
tool_execution> Tool:getOpportunities Args:{'id': 'acme'}
tool_execution> Tool:getOpportunities Response:{"type":"text","text":"Active Opportunities: {\n  \"opportunity_id\": 1,\n  \"status\": \"active\",\n  \"account_id\": 1,\n  \"account_name\": \"Acme Corp\",\n  \"item_id\": 2,\n  \"description\": \"Upsell - Cloud package\",\n  \"amount\": \"5000.00\",\n  \"year\": 2025\n}","annotations":null}
inference> The active opportunities for the account "Acme" are:

* Opportunity ID: 1
* Status: Active
* Account ID: 1
* Account Name: Acme Corp
* Item ID: 2
* Description: Upsell - Cloud package
* Amount: $5000.00
* Year: 2025


### Output Analysis

This script defines and runs a simple agent using Llama Stack that is equipped with CRM tools (mcp::crm). It performs a single action: retrieves one or more active opportunities using a user prompt. The response is streamed, and each part of the process (model inference, tool call, and results) is logged step-by-step.

Output: 

The model infers that it should call the getOpportunities tool with argument id=1.

This suggests the tool is expected to return the active opportunity with ID 1.

The tool returns a JSON-like structure containing multiple active opportunities.

The model interprets the output and summarizes only one opportunity (ID 1), including:

Account: Acme Corp

Description: "Upsell - Cloud package"

Amount: $5,000

Year: 2025


## Analyze account sentiment for active opportunities

Get a list of active opportunities, then find the associated support cases for each opportunity, then anayse the mood of the account.

In [6]:
# Here is a system prompt we have come up with which works well for this query
sys_prompt2="""You are a helpful AI assistant, responsible for helping me find and communicate information back to my team.
    You have access to a number of tools.
    Whenever a tool is called, be sure return the Response in a friendly and helpful tone.
    When you are asked to find out about opportunities and support casescounts you must use a tool.
    If you need to create a pdf, add this markdown to the start of the content:  ![ParasolCloud Logo](https://i.postimg.cc/MHZB5tmL/Screenshot-2025-04-21-at-5-58-46-PM.png)
*Secure Cloud Solutions for a Brighter Business*  
---
    """

In [25]:
from llama_stack_client.lib.agents.agent import Agent

agent = Agent(
    client=client,
    model=model,
    instructions=sys_prompt2,
    tools=["mcp::crm", "mcp::pdf"],
    tool_config={"tool_choice":"auto"},
    sampling_params={
        "max_tokens":4096,
        "strategy": {"type": "greedy"},
    }
)

session_id = agent.create_session(session_name="active_opportunity_and_account_health")

prompts = [
    """Get active opportunities for ACME""",
    """get a list of support cases for the account id""",
    """Analyze the support cases and determine how happy the account is""",
     """Create a pdf with the content 'hello world'""",

]
for i, prompt in enumerate(prompts):    
    turn_response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=True,
    )
    logger.info(f"========= Turn: {i} =========")
    for log in EventLogger().log(turn_response):
        log.print()


========= Turn: 0 =========


inference> 
tool_execution> Tool:getOpportunities Args:{'company': 'ACME'}
tool_execution> Tool:getOpportunities Response:{"type":"text","text":"Active Opportunities: {\n  \"opportunity_id\": 1,\n  \"status\": \"active\",\n  \"account_id\": 1,\n  \"account_name\": \"Acme Corp\",\n  \"item_id\": 2,\n  \"description\": \"Upsell - Cloud package\",\n  \"amount\": \"5000.00\",\n  \"year\": 2025\n}","annotations":null}
inference> The active opportunities for ACME are:

* Opportunity ID: 1
* Status: Active
* Account ID: 1
* Account Name: Acme Corp
* Item ID: 2
* Description: Upsell - Cloud package
* Amount: $5000.00
* Year: 2025


========= Turn: 1 =========


inference> 
tool_execution> Tool:getSupportCases Args:{'account_id': '1'}
tool_execution> Tool:getSupportCases Response:{"type":"text","text":"Support Cases for account i:[\n  {\n    \"case_id\": 1,\n    \"subject\": \"Login failure\",\n    \"description\": \"Customer unable to log in with correct credentials.\",\n    \"status\": \"open\",\n    \"severity\": \"High\",\n    \"created_at\": \"2025-04-16T22:57:50.901Z\",\n    \"account_name\": \"Acme Corp\"\n  },\n  {\n    \"case_id\": 2,\n    \"subject\": \"Slow dashboard\",\n    \"description\": \"Performance issues loading analytics dashboard.\",\n    \"status\": \"in progress\",\n    \"severity\": \"Critical\",\n    \"created_at\": \"2025-04-16T22:57:50.901Z\",\n    \"account_name\": \"Acme Corp\"\n  },\n  {\n    \"case_id\": 6,\n    \"subject\": \"Feature request: Dark mode\",\n    \"description\": \"Request to implement dark mode UI.\",\n    \"status\": \"closed\",\n    \"severity\": \"Low\",\n    \"created_at\": \"2025-04-16T22:57:

========= Turn: 2 =========


inference> Based on the support cases, I can analyze the account's satisfaction level. Here's my assessment:

**Positive indicators:**

* Only one case (Case ID: 6) is marked as "Closed" with a severity of "Low", which suggests that the account is not extremely dissatisfied.
* The other two cases (Cases ID: 1 and 2) are still open, but they are marked as "Open" and "In Progress", respectively, which indicates that the support team is actively working on resolving the issues.

**Negative indicators:**

* Two cases (Cases ID: 1 and 2) are related to performance issues (login failure and slow dashboard), which can impact the account's productivity and overall satisfaction.
* One case (Case ID: 2) is marked as "Critical", which suggests that the issue is significant and may have a substantial impact on the account's operations.

**Overall assessment:**

Based on the analysis, I would rate the account's satisfaction level as 6 out of 10. The account is experiencing some issues, but the supp

========= Turn: 3 =========


inference> 
tool_execution> Tool:create_pdf_from_markdown Args:{'markdown': 'hello world', 'outputFilename': 'output.pdf', 'paperFormat': 'letter', 'paperOrientation': 'portrait', 'paperBorder': '2cm', 'watermark': ''}
tool_execution> Tool:create_pdf_from_markdown Response:{"type":"text","text":"Successfully created PDF at: /Users/phayes/output.pdf","annotations":null}
inference> The PDF with the content 'hello world' has been successfully created and saved to the output directory.


#### Output Analysis

`tool_execution> Tool:getSupportCases Args:{'id': '1'}`

The model identifies account_id: 1 (from the previous opportunity) and passes it to the getSupportCases tool.

<b>Tool Response:</b>
Returns 3 support cases for Acme Corp, each with detailed metadata.

<b>Model's Inference:</b>
Extracts and formats all 3 support cases clearly.

Each case is displayed with its:

* Subject
* Description
* Status
* Severity
* Created timestamp

The agent performs a sentiment analysis over the support cases to determine the state of the account and makes recommendations.


## Key Takeaways

This tutorial demonstrates how to build agentic MCP applications with Llama Stack. We do so by initializing an agent while giving it access to the MCP tools configured with Llama Stack, then invoking the agent on each of the specified queries. 

In [74]:
from llama_stack_client import Agent
# Create simple agent with tools
sys_prompt1= """You are a helpful assistant. Use tools to answer."""
agent = Agent(
    client,
    model="meta-llama/Llama-3.2-3B-Instruct", # replace this with your choice of model
    instructions = sys_prompt1 , # update system prompt based on the model you are using
    tools=["mcp::python"],
    tool_config={"tool_choice":"auto"},
    sampling_params={
        "max_tokens":10000,
        "strategy": {"type": "greedy"},
    }
)

user_prompts = ["Find the number of letter s in the word strawberry"]
session_id = agent.create_session(session_name="OCP_demo")

for prompt in user_prompts:
    turn_response = agent.create_turn(
        messages=[
            {
                "role":"user",
                "content": prompt
            }
        ],
        session_id=session_id,
        stream=True,
    )
    for log in EventLogger().log(turn_response):
        log.print()

inference> 
